In [1]:
import os
import requests
sys.path.append(os.path.normpath('../morpc-parcel-fetch/'))
import morpcParcels

In [2]:
parcel_urls = {
    "Delaware":"https://services2.arcgis.com/ziXVKVy3BiopMCCU/arcgis/rest/services/Parcel/FeatureServer/0",
    "Pickaway":"https://services6.arcgis.com/FhJ42byMw3LmPYCN/arcgis/rest/services/Parcels_Search/FeatureServer/1",
    "Logan":"https://services9.arcgis.com/mFxO7gBbusFBQ5o9/ArcGIS/rest/services/Logan_County_Parcels/FeatureServer/12",
    "Ross":"https://services7.arcgis.com/IQSUQhVBDHAkRlWe/ArcGIS/rest/services/parcel_joined/FeatureServer/0"
}

In [3]:
morpcParcels.gdf_from_services(url = parcel_urls['Delaware'], fieldIds=["CLASS","YRBUILT","ACRES"])

https://services2.arcgis.com/ziXVKVy3BiopMCCU/arcgis/rest/services/Parcel/FeatureServer/0/query?outFields=CLASS,YRBUILT,ACRES&where=1%3D1&f=geojson


,geometry,CLASS,YRBUILT,ACRES
0,"POLYGON ((-83.19830 40.44092, -83.20534 40.441...",501,0,3.966
1,"POLYGON ((-83.19211 40.44038, -83.19404 40.440...",501,0,0.850
2,"POLYGON ((-83.19411 40.43999, -83.19413 40.439...",511,1987,1.898
3,"POLYGON ((-83.19288 40.43941, -83.19421 40.439...",511,1950,2.280
4,"POLYGON ((-83.19582 40.44010, -83.19576 40.440...",511,2015,1.492
...,...,...,...,...
83,"POLYGON ((-83.14948 40.38082, -83.14959 40.380...",455,1920,0.530
84,"POLYGON ((-83.14865 40.37766, -83.14876 40.377...",499,1996,5.869
85,"POLYGON ((-83.14865 40.37766, -83.14876 40.377...",499,1997,5.869
86,"POLYGON ((-83.14903 40.38385, -83.14926 40.383...",499,1920,0.000
